In [2]:
import numpy as np
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn import svm
import xgboost as xgb
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.ensemble import VotingRegressor, VotingClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [4]:
df =  pd.read_pickle('../data/dataframes/unificated_data_set.pkl')
df.head()

,blue_ribbon_award,golden_gross,hochi_eigashou,kinejun_best_ten,mainichi_film_award,nikkan_sports,中井貴一,丹波哲郎,井川比佐志,仲代達矢,...,澤井信一郎,田中陽造,神波史男,筒井ともみ,荒井晴彦,那須真知子,鄭義信,screen_time,year,prize
id,,,,,,,,,,,,,,,,,,,,,
1,-0.144358,-0.186589,-0.145248,-0.190394,-0.190618,-0.19017,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-1.131031,1978,1
2,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.19017,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,2.242866,1978,0
3,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.19017,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.048838,1978,0
4,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.19017,-0.187707,-0.141691,-0.188379,-0.187036,...,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,2.752133,1978,0
5,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.19017,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.558105,1978,0


In [8]:
train = df.loc[df["year"] != 2019]
y_train = train['prize'].ravel()
x_train = train.drop(['prize', 'year'], axis=1)

In [9]:
x_train

,blue_ribbon_award,golden_gross,hochi_eigashou,kinejun_best_ten,mainichi_film_award,nikkan_sports,中井貴一,丹波哲郎,井川比佐志,仲代達矢,...,李相日,松田寛夫,澤井信一郎,田中陽造,神波史男,筒井ともみ,荒井晴彦,那須真知子,鄭義信,screen_time
id,,,,,,,,,,,,,,,,,,,,,
1,-0.144358,-0.186589,-0.145248,-0.190394,-0.190618,-0.190170,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-1.131031
2,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.190170,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,2.242866
3,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.190170,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.048838
4,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.190170,-0.187707,-0.141691,-0.188379,-0.187036,...,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,2.752133
5,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.190170,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.558105
6,-0.190170,-0.186589,-0.191067,-0.190394,-0.144803,-0.190170,-0.187707,-0.141691,-0.188379,-0.187036,...,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,0.778722
7,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.190170,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-1.131031
8,-0.190170,-0.186589,-0.191067,-0.190394,-0.190618,-0.190170,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-1.003715
9,-0.144358,-0.186589,-0.145248,-0.144580,-0.190618,-0.190170,-0.187707,-0.187483,-0.188379,-0.187036,...,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-0.185694,-0.185471,-0.185471,-0.185471,-1.449324


In [5]:
def load_data(df, test_year):
    
    train = df.loc[df["year"] != test_year]
    test = df.loc[df["year"] == test_year]
    
    # 入力データの作成
    y_train = train['prize'].ravel()
    x_train = train.drop(['prize', 'year'], axis=1)
    
    features = x_train.columns
    
    x_train = x_train.values # 学習データ
    
    # テストデータの作成
    y_test = test['prize'].ravel()
    x_test = test.drop(['prize', 'year'], axis=1)
    x_test = x_test.values # テストデータ
    
    return x_train, x_test, y_train, y_test, features

In [6]:
def plot_roc_curve(fpr, tpr, auc):
    # ROC曲線をプロット
    plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
    plt.legend()
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.show()

In [7]:
def create_models():
    lr = LogisticRegression(class_weight="balanced", solver="liblinear",  penalty="l2", C=0.0001)
    
    sv = svm.SVR(kernel="poly",
                                 degree=8,
                                 gamma=0.03521735642853326,
                                 coef0=0.34010389238140537,
                                 tol=1e-05,
                                 C=0.001,
                                 epsilon=0.14620884632948022)

    xgboost = xgb.XGBRegressor(silent= True, 
                           max_depth=1,
                           learning_rate=0.14544434403253392,
                           n_estimators = 72,
                           gamma=0.4356018082020117,
                           reg_lambda=2.931451663505623,
                           reg_alpha=0.19045302677956732)

    lgbm = lgb.LGBMRegressor(
        verbosity=1,
        boost_from_average='true',
        metric='auc',
        bagging_freq=5,
        max_depth=19,
        learning_rate=0.09802221664336347,
        n_estimators=703,
        subsample=0.7596658467659505,
        reg_lambda=0.4087114877863393,  
    )
    
    lda = LDA(solver="eigen", shrinkage=1)

    estimators_rgs = [('sv', sv), ('xgboost', xgboost),('lgbm', lgbm)]
    
    return estimators_rgs

In [8]:
def main():
    
    df =  pd.read_pickle('../data/dataframes/unificated_data_set.pkl')
    
    cm_all = np.zeros((2, 2))
    
    # 予測した確率全体を格納
    probs_all = np.array([])
    y_true_all = np.array([])
    
    for y in range(1978, 2020):
        
        # データの生成
        x_train, x_test, y_train, y_test, features = load_data(df, y)
       
        # 学習
        estimators= create_models()
        
        vr = VotingRegressor(estimators)

        vr.fit(x_train, y_train)
        
        # 予測(確率)
        probs = vr.predict(x_test)
        probs_all = np.hstack((probs_all, probs))
        y_true_all = np.hstack((y_true_all, y_test))
        
        print(probs)

        # 混同行列
        y_pred = np.where((probs ==  max(probs)), 1, 0) #確率→0/1
        cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
        cm_all += cm

    auc = roc_auc_score(y_true_all, probs_all)
    fpr, tpr, thresholds = roc_curve(y_true_all, probs_all)
    
    plot_roc_curve(fpr, tpr, auc)
    
    print("len: {0} , {1}".format(len(y_true_all), len(probs_all) ))
    print("confusion_matrix: ")
    print(cm_all)
    print("AUC: ")
    print(auc)
    print()
    return

In [ ]:
main()

[0.14404603 0.0371832  0.11007778 0.05628059 0.10894265]
[0.2221941  0.08417103 0.02108253 0.1106934  0.21092238]
[0.24007355 0.31097529 0.08657966 0.38967664 0.10109638]
